# Introduction to Python IV: OOP, iterators, and generators

## Content
- What are classes and how to use/build them?
- What are iterators and generators and what's the difference?
- Some thoughts about testing...

## Remember jupyter notebooks
- To run the currently highlighted cell, hold <kbd>&#x21E7; Shift</kbd> and press <kbd>&#x23ce; Enter</kbd>.
- To get help for a specific function, place the cursor within the function's brackets, hold <kbd>&#x21E7; Shift</kbd>, and press <kbd>&#x21E5; Tab</kbd>.

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

## Object-oriented programming (OOP)
A `class` is a blueprint for a container like data structure containing variables (**attributes**) and functions (**methods**). An `object` is an actual instance of a `class`.

**Example**: let's build a rotation matrix, i.e.,

$$\mathbf{R}(\phi) = \begin{pmatrix} \cos(\phi) & -\sin(\phi) \\ \sin(\phi) & \cos(\phi) \end{pmatrix},$$

which rotates a vector $\mathbf{r}\in\mathbb{R}^2$ by the angle $\phi$.

As a function, it could look like this:

In [2]:
def rotate(vector, angle):
    """Rotate the given vector by the given angle"""
    rad = np.pi * angle / 180
    rotation = np.array([
        [np.cos(rad), -np.sin(rad)],
        [np.sin(rad), np.cos(rad)]])
    return np.dot(rotation, np.asarray(vector))


print(rotate([1, 0], 90))

[6.123234e-17 1.000000e+00]


Now, let's build a class to encapsulate this behaviour!

In [3]:
class RotationMatrix(object):
    """This will rotate a 2D vector by some angle phi"""
    
    def __init__(self, angle):
        """
        Arguments:
            angle (float): angle in °
        """
        self.angle = angle % 360


rotation = RotationMatrix(90)
print(rotation.angle)
print(rotation)

90


Printing `rotation` tells us that it is an **object** as well as its address &mdash; but nothing more. To learn about its actual angle, we need to look up the `angle` **attribute**.

In [4]:
class RotationMatrix(object):
    """This will rotate a 2D vector by some angle phi"""
    
    def __init__(self, angle):
        """
        Arguments:
            angle (float): angle in °
        """
        self.angle = angle % 360

    def __repr__(self):
        """Represent the object by some informative string"""
        return f'Rotation by {self.angle:.2f}°'

    
rotation = RotationMatrix(90)
print(rotation)

Rotation by 90.00°


The `__repr__` method is one of many **magic methods** in Python. These methods have two leading and trailing underscores and are called in very specific circumstances, e.g., when we **initialise** the object (`__init__`) or if we attempt to **print** the object (`__repr__`).

In [5]:
rotation2 = RotationMatrix(-90)
print(rotation2)

Rotation by 270.00°


Much better! But remember that our class should represent a **matrix**, and matrices can be multiplied...

In [6]:
rotation * rotation2

TypeError: unsupported operand type(s) for *: 'RotationMatrix' and 'RotationMatrix'

We want to make matrix multiplication available. For that, we need another **magic method**, `__mul__`:

In [7]:
class RotationMatrix(object):
    """This will rotate a 2D vector by some angle phi"""
    
    def __init__(self, angle):
        """
        Arguments:
            angle (float): angle in °
        """
        self.angle = angle % 360

    def __repr__(self):
        """Represent the object by some informative string"""
        return f'Rotation by {self.angle:.2f}°'
    
    def __mul__(self, other):
        """Multiply two rotations by adding their angles"""
        return RotationMatrix(self.angle + other.angle)


rotation = RotationMatrix(90)
print(rotation)

rotation2 = RotationMatrix(-90)
print(rotation2)

print(rotation * rotation2)
print(rotation.__mul__(rotation2))

Rotation by 90.00°
Rotation by 270.00°
Rotation by 0.00°
Rotation by 0.00°


Here, we have used that a product of two rotation matrices is another rotation matrix where the angle is the sum of the two original angles.

Note that we now have (rotation) matrix multiplication available without even having implemented the actual rotation of a vector...

In [8]:
class RotationMatrix(object):
    """This will rotate a 2D vector by some angle phi"""
    
    def __init__(self, angle):
        """
        Arguments:
            angle (float): angle in °
        """
        self.angle = angle % 360
        self.rad = np.pi * self.angle / 180
        self.rotation_matrix = np.array([
            [np.cos(self.rad), -np.sin(self.rad)],
            [np.sin(self.rad), np.cos(self.rad)]])
        
    def __repr__(self):
        """Represent the object by some informative string"""
        return f'Rotation by {self.angle:.2f}°'
    
    def __mul__(self, other):
        """Multiply two rotations by adding their angles"""
        return RotationMatrix(self.angle + other.angle)

    def rotate(self, vector):
        """Rotate the vector"""
        return np.dot(self.rotation_matrix, np.asarray(vector))


rotation = RotationMatrix(90)
print(rotation.rotate([1, 0]))

[6.123234e-17 1.000000e+00]


And now

In [9]:
class RotationMatrix2(RotationMatrix):
    """This is an example on inheritance"""

    def __mul__(self, other):
        """Multiply two rotations by adding their angles"""
        return RotationMatrix2(self.angle + other.angle)

    def __call__(self, vector):
        """Rotate the vector"""
        return self.rotate(vector)


r1 = RotationMatrix2(180)
r2 = RotationMatrix(-90)

print(isinstance(r1, RotationMatrix), isinstance(r1, RotationMatrix2))
print(isinstance(r2, RotationMatrix), isinstance(r2, RotationMatrix2))

True True
True False


In [10]:
print(isinstance(r1 * r2, RotationMatrix), isinstance(r1 * r2, RotationMatrix2))
print(isinstance(r2 * r1, RotationMatrix), isinstance(r2 * r1, RotationMatrix2))

True True
True False


In [11]:
print(r1 * r2)
print((r1 * r2)([1, 0]))

Rotation by 90.00°
[6.123234e-17 1.000000e+00]


**Example**: Some of Python's **magic methods** and their corresponding binary/unary operators:

In [12]:
class Tester(object):
    def __add__(self, value):
        print(f'__add__({value})')
    def __sub__(self, value):
        print(f'__sub__({value})')
    def __mul__(self, value):
        print(f'__mul__({value})')
    def __truediv__(self, value):
        print(f'__truediv__({value})')
    def __floordiv__(self, value):
        print(f'__floordiv__({value})')
    def __mod__(self, value):
        print(f'__mod__({value})')
    def __pow__(self, value):
        print(f'__pow__({value})')
    def __and__(self, value):
        print(f'__and__({value})')
    def __or__(self, value):
        print(f'__or__({value})')
    def __xor__(self, value):
        print(f'__xor__({value})')
    def __eq__(self, value):
        print(f'__eq__({value})')
    def __neq__(self, value):
        print(f'__neq__({value})')
    def __gt__(self, value):
        print(f'__gt__({value})')
    def __ge__(self, value):
        print(f'__ge__({value})')
    def __lt__(self, value):
        print(f'__lt__({value})')
    def __le__(self, value):
        print(f'__le__({value})')
    def __radd__(self, value):
        print(f'__radd__({value})')
    def __iadd__(self, value):
        print(f'__iadd__({value})')


t = Tester()
t + 1
t - 1
t * 1
t / 1
t // 1
t % 1
t ** 1
t & 1
t | 1
t ^ 1
t == 1
t != 1
t > 1
t >= 1
t < 1
t <= 1
1 + t
t += 1

__add__(1)
__sub__(1)
__mul__(1)
__truediv__(1)
__floordiv__(1)
__mod__(1)
__pow__(1)
__and__(1)
__or__(1)
__xor__(1)
__eq__(1)
__eq__(1)
__gt__(1)
__ge__(1)
__lt__(1)
__le__(1)
__radd__(1)
__iadd__(1)


## Iterators and generators
We revisit the Fibonacci problem:

$$f_i = f_{i-1} + f_{i-2},\quad i\geq2,\, f_0=f_1=1.$$


In the second introduction notebook, we *solved* this problem with the code snippet

```Python
a, b = 1, 1
while True:
    a, b = a + b, a
    if a < 100:
        print(a)
    else:
        break
```

Now, we create an `iterator` which we could use in a `for` loop:

In [13]:
class Fibonacci(object):
    """An iterator over Fibonacci numbers"""
    
    def __init__(self, stop):
        """Arguments:
            stop (int): endpoint for the iteration
        """
        self.stop = stop
        self.a, self.b = 1, 1
        
    def __iter__(self):
        """This is what make this class an iterator"""
        return self
    
    def __next__(self):
        """This is what actually computes the next number"""
        self.a, self.b = self.a + self.b, self.a
        if self.a < self.stop:
            return self.a
        raise StopIteration


fib = Fibonacci(100)
print(fib)

In [14]:
for value in fib:
    print(value)

2
3
5
8
13
21
34
55
89


In [15]:
print(list(Fibonacci(100)))

[2, 3, 5, 8, 13, 21, 34, 55, 89]


In [16]:
fib = Fibonacci(100)
for i in range(70):
    print(i, next(fib))

0 2
1 3
2 5
3 8
4 13
5 21
6 34
7 55
8 89


StopIteration: 

Hence, an `iterator` is a `class` which has the methods `__iter__()` and `__next__()`.

A `generator` behaves very similar but does not require to write a `class`. The key component here is the `yield` command:

In [17]:
def fibonacci(stop):
    """Generate Fibonacci numbers
    
    Arguments:
        stop (int): endpoint for the generation
    """
    a, b = 1, 1
    while True:
        a, b = a + b, a
        if a < stop:
            yield a
        else:
            break


print(fibonacci(100))

<generator object fibonacci at 0x11aace7c8>


In [18]:
for value in fibonacci(100):
    print(value)

2
3
5
8
13
21
34
55
89


In [19]:
print(list(fibonacci(100)))

[2, 3, 5, 8, 13, 21, 34, 55, 89]


In [20]:
fib = fibonacci(100)
for i in range(10):
    print(i, next(fib))

0 2
1 3
2 5
3 8
4 13
5 21
6 34
7 55
8 89


StopIteration: 

## Some thoughts about testing...

When using the **test-driven development** pattern, we want to make sure that testing each piece of code plays a central role in the entire development process. The cycle reads

1. define a new interface (function, class, ...) changed functionality
2. design conditions under which you can predict the behaviour
3. write tests to check these predictions
4. implement a quick prototype which passes the tests
5. refactor the prototype for readability, speed, ...
6. start over at 1.

In practise, we can use `assert` to test predicted behaviour:

In [21]:
assert True, 'This is True and, thus, not interesting!'

In [22]:
assert False, 'This is False and very interesting!'

AssertionError: This is False and very interesting!

**Example**: we want to write a generator which yields all neighbouring intergers up to its (excluded) stopping point, i.e.,

$$[0, 1, 2, 3, 4, ..., n - 1] \quad \to \quad [(0, 1), (1, 2), (2, 3), ..., (n - 2, n - 1)]$$

for the use case `pairs(n)`.

We start the development process by writing some test function and the interface and convince ourselfs that the test fails:

In [23]:
def test_pairs(generator):
    for x, y in generator:
        assert y == x + 1, f'Failure for ({x}, {y})'


def pairs(stop):
    pass


test_pairs(pairs(100))

TypeError: 'NoneType' object is not iterable

Now we start prototyping and check the test after every change:

In [24]:
def pairs(stop):
    for x in range(stop):
        yield x


test_pairs(pairs(100))

TypeError: cannot unpack non-iterable int object

Like for the empty interface, the `assert` is not even evaluated. This, too, is an important test result.

In [25]:
def pairs(stop):
    for x in range(stop):
        yield x, x


test_pairs(pairs(100))

AssertionError: Failure for (0, 0)

Now we can evaluate the pairs, but they are incorrect.

In [26]:
def pairs(stop):
    for x in range(stop):
        yield x, x + 1


test_pairs(pairs(100))

No traceback, no exception, great! Are we done?

No, because our test was not really exhaustive and has neglected some part of the interface specifications. We, thus, need better tests!

In [27]:
list(pairs(5))

[(0, 1), (1, 2), (2, 3), (3, 4), (4, 5)]

In a test suite, you often find test functions which bundle a number of tests for a specific function, testing various aspects of the function's behaviour:

In [29]:
def test_pair_generator():
    test_pairs(pairs(100))
    assert len(list(pairs(100))) == 99, 'Generator too long'


test_pair_generator()

AssertionError: Generator too long

And, finally,

In [30]:
def pairs(stop):
    for x in range(stop - 1):
        yield x, x + 1


test_pair_generator()

we have something which **might** actually work as we intend it to work.

If you want to write good tests, don't give in to **confirmation bias** just check that your function gives you the expected result for very few/easy cases. You should try to actually **break** your function, feed it improper parameters, and check that it breaks down where it should.

In [31]:
import random

In [34]:
random.seed(0)
print(np.random.rand(3))

random.seed(0)
print(np.random.rand(3))

[0.46632318 0.40476434 0.51524556]
[0.62020487 0.57704803 0.42317815]


In [35]:
np.random.seed(0)
print(np.random.rand(3))

np.random.seed(0)
print(np.random.rand(3))

[0.5488135  0.71518937 0.60276338]
[0.5488135  0.71518937 0.60276338]
